In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [3]:
m = nn.utils.spectral_norm(nn.Conv2d(64, 128, kernel_size=1, bias=False, groups=1))
input = torch.randn(1, 64, 32, 32)
out = m(input)
out[0, 1, :5, :5]

tensor([[-0.4825, -0.1248,  0.6937,  0.2335, -0.8205],
        [-0.8891, -0.6662, -0.0029,  0.6946,  0.0114],
        [ 1.1126, -0.2605, -0.3140, -0.0830, -0.2213],
        [ 0.0820, -0.1357, -0.5066, -0.0305, -0.1445],
        [-0.7284, -0.4668,  0.1604, -0.4937,  0.1338]],
       grad_fn=<SliceBackward0>)

In [ ]:

class CustomConv2d(nn.Conv2d):
    def __init__(self, *args, scaling_factor_init=0.5, **kwargs):
        # Call the constructor of the parent class (nn.Conv2d)
        super(CustomConv2d, self).__init__(*args, **kwargs)
        self.scaling_factor = nn.Parameter(torch.ones((1, args[1], 1, 1), dtype=torch.float32) * scaling_factor_init) # Add a custom attribute



    def forward(self, x):
        # Perform the standard convolution operation
        conv_output = super(CustomConv2d, self).forward(x)

        norm_factor = F.conv2d(
            torch.ones_like(x),
            torch.abs(self.weight),
            self.bias,
            self.stride, self.padding,
            self.dilation, self.groups
        )
        output = (conv_output) / (norm_factor * self.scaling_factor)
        return output
        

In [11]:
m = CustomConv2d(64, 128, kernel_size=1, bias=False, groups=1)
input = torch.randn(1, 64, 32, 32)
out = m(input)
out[0, 1, :5, :5]

conv_output=torch.Size([1, 128, 32, 32]) 

norm_factor=torch.Size([1, 128, 32, 32]) 



tensor([[-0.4274, -0.3207, -0.3562,  0.0508, -0.0410],
        [ 0.1633, -0.0317, -0.2028,  0.1405,  0.3640],
        [-0.0438, -0.0680, -0.1252, -0.3507,  0.1938],
        [ 0.1856,  0.1667, -0.1639, -0.4654,  0.0263],
        [-0.2193, -0.2813,  0.3328, -0.0202, -0.2113]],
       grad_fn=<SliceBackward0>)

In [9]:
m.scaling_factor.shape

torch.Size([1, 128, 1, 1])